In [17]:
import pandas as pd
import numpy as np

path = 'MovieSummaries\\'
df_plot_summaries = pd.read_csv('MovieSummaries/plot_summaries.txt', sep='\t', names = ['WikipediaMovieID', 'PlotSummaries'], index_col = ['WikipediaMovieID'])
df_character = pd.read_csv("MovieSummaries/character.metadata.tsv", sep='\t', names = ['WikipediaMovieID', 'FreebaseMovieID', 'MovieReleaseDate', 'CharacterName', 
                                                                     'ActorDateOfBirth', 'ActorGender', 'ActorHeight', 'ActorEthnicity', 
                                                                     'ActorName', 'ActorAgeAtMovieRelease','FreebaseCharacter', 
                                                                     'FreebaseCharacterID', 'FreebaseActorID'], index_col = ['WikipediaMovieID'])
df_movies = pd.read_csv("MovieSummaries/movie.metadata.tsv", sep='\t', header = None, names = ['WikipediaMovieID', 'FreebaseMovieID', 'MovieName', 
                                                                                       'MovieReleaseDate', 'MovieBoxOfficeRevenue',
                                                                                       'MovieRuntime', 'MovieLanguages', 'MovieCountries',
                                                                                       'MovieGenres'], index_col = ['WikipediaMovieID'])


df_movies['MovieReleaseDate'] = pd.to_datetime(df_movies['MovieReleaseDate'], format='%Y-%m-%d', errors='coerce').dt.year.astype('Int64')
df_movies = pd.merge(df_movies, df_plot_summaries, how='left', on ='WikipediaMovieID')

# IMDB non-commercial dataset downloaded from https://developer.imdb.com/non-commercial-datasets/.
df_basics = pd.read_csv("title.basics.tsv/data.tsv", sep='\t')
df_basics = df_basics[df_basics['titleType'] == 'movie']
df_ratings = pd.read_csv("title.ratings.tsv/data.tsv", sep='\t')


df_IMDb_merged = pd.merge(df_basics, df_ratings, how='right', on='tconst')
df_IMDb_merged['MovieName'] = df_IMDb_merged['primaryTitle']
df_IMDb_merged.rename(columns={'startYear':'MovieReleaseDate'}, inplace =True)
df_IMDb_merged.rename(columns={'tconst':'IMDbID'}, inplace=True)
df_IMDb_merged['MovieReleaseDate'] = pd.to_datetime(df_IMDb_merged['MovieReleaseDate'], errors='coerce').dt.year.astype('Int64')



df = df_movies.merge(df_IMDb_merged[['IMDbID', 'MovieName', 'averageRating', 'primaryTitle', 'MovieReleaseDate', 'genres']], how= 'outer', on = ['MovieName', 'MovieReleaseDate'])
df.drop(['FreebaseMovieID', 'MovieGenres', 'primaryTitle'], axis=1, inplace=True)
new_column_order = ['IMDbID'] + [col for col in df.columns if col != 'IMDbID']
df = df[new_column_order]
df.to_csv('merged_cmu_imdb.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'title.basics.tsv/data.tsv'

In [12]:
import pandas as pd
import numpy as np
import spacy


df_plot_summaries = pd.read_csv('MovieSummaries/plot_summaries.txt', sep='\t', names = ['WikipediaMovieID', 'PlotSummaries'], index_col = ['WikipediaMovieID'])
"""    df_character = pd.read_csv("MovieSummaries/character.metadata.tsv", sep='\t', names = ['WikipediaMovieID', 'FreebaseMovieID', 'MovieReleaseDate', 'CharacterName', 
                                                                    'ActorDateOfBirth', 'ActorGender', 'ActorHeight', 'ActorEthnicity', 
                                                                    'ActorName', 'ActorAgeAtMovieRelease','FreebaseCharacter', 
                                                                    'FreebaseCharacterID', 'FreebaseActorID'], index_col = ['WikipediaMovieID'])"""
df_movies = pd.read_csv("MovieSummaries/movie.metadata.tsv", sep='\t', header = None, names = ['WikipediaMovieID', 'FreebaseMovieID', 'MovieName', 
                                                                                    'MovieReleaseDate', 'MovieBoxOfficeRevenue',
                                                                                    'MovieRuntime', 'MovieLanguages', 'MovieCountries',
                                                                                    'MovieGenres'], index_col = ['WikipediaMovieID'])


df_movies['MovieReleaseDate'] = pd.to_datetime(df_movies['MovieReleaseDate'], format='%Y-%m-%d', errors='coerce').dt.year.astype('Int64')
df_movies = pd.merge(df_movies, df_plot_summaries, how='left', on ='WikipediaMovieID')


# Load the CSV files into pandas DataFrames
rating = pd.read_csv('data/ratings.csv')
links = pd.read_csv('data/links.csv')
movies = pd.read_csv('data/movies_metadata.csv', low_memory=False)


# Load spaCy model with disabled components for efficiency
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Pre-compile the list of stop words for faster checking
stop_words = nlp.Defaults.stop_words

# Function to check if token should be included
def is_token_allowed(token):
    return not token.is_punct and token.pos_ in ["ADJ", "PROPN", "NOUN"] and token.text.lower() not in stop_words

# Function to preprocess token
def preprocess_token(token):
    return token.lemma_.strip().lower()

def process_text(summary):
    # Ensure the text is treated as a writable string
    doc = nlp(summary)
    # Convert the processed tokens to strings to avoid any complex types
    return [preprocess_token(token) for token in doc if is_token_allowed(token)]

# Group by 'id' and calculate the average rating for each group
average_ratings = rating.groupby('movieId')['rating'].agg(np.nanmean).reset_index()

def strip(text):
    return str(text).replace('tt', '')

# Merge DataFrames on the "id" column
merged_df = pd.merge(average_ratings, links, on='movieId', how='outer')
merged_df.rename(columns={'imdbId': 'imdb_id'}, inplace=True)
merged_df.imdb_id = merged_df['imdb_id']
# Change types

merged_df['imdb_id'] = merged_df['imdb_id'].astype(str)
movies['imdb_id'] = movies['imdb_id'].apply(strip).astype(str)
movies = movies.astype(str)

merged_df = pd.merge(merged_df, movies, on='imdb_id', how='inner')

#df_plot_summaries['tokens']= df_plot_summaries['Summary'].apply(process_text)

#pd.merge(merged_df, df_plot_summaries, on='Wikipedia_ID', how='left')

merged_df.release_date = pd.to_datetime(merged_df.release_date, errors='coerce')

# Now extract the year
merged_df.release_date = merged_df.release_date.dt.year.astype("Int64")
merged_df.release_date


columns = ['FreebaseMovieID', 'original_title', 'release_date',
       'MovieBoxOfficeRevenue', 'MovieRuntime', 'MovieLanguages',
       'MovieCountries', 'MovieGenres', 'PlotSummaries']
df_movies.columns = columns


merged_df = pd.merge(merged_df, df_movies, how= 'outer', on =['original_title', 'release_date'])
print(merged_df)
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('data/merged_file.csv', index=False)

0        2007-05-21
1        2007-04-06
2        2007-09-07
3        2007-08-24
4        2007-11-01
            ...    
15404    2015-04-24
15405    1963-12-06
15406    2016-11-11
15407    2017-08-04
15408    2017-06-09
Name: release_date, Length: 15409, dtype: object

In [22]:
merged_df.release_date

#df_plot_summaries['tokens']= df_plot_summaries['Summary'].apply(process_text)

#pd.merge(merged_df, df_plot_summaries, on='Wikipedia_ID', how='left')
merged_df.release_date = pd.to_datetime(merged_df.release_date, errors='coerce')

# Now extract the year
merged_df.release_date = merged_df.release_date.dt.year.astype("Int64")
merged_df.release_date


columns = ['FreebaseMovieID', 'original_title', 'release_date',
       'MovieBoxOfficeRevenue', 'MovieRuntime', 'MovieLanguages',
       'MovieCountries', 'MovieGenres', 'PlotSummaries']
df_movies.columns = columns


merged_df = pd.merge(merged_df, df_movies, how= 'outer', on =['original_title', 'release_date'])
print(merged_df)
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('data/merged_file.csv', index=False)


0        2007
1        2007
2        2007
3        2007
4        2007
         ... 
15404    2015
15405    1963
15406    2016
15407    2017
15408    2017
Name: release_date, Length: 15409, dtype: Int64

In [18]:
df_movies.columns


Index(['FreebaseMovieID', 'MovieName', 'MovieReleaseDate',
       'MovieBoxOfficeRevenue', 'MovieRuntime', 'MovieLanguages',
       'MovieCountries', 'MovieGenres', 'PlotSummaries'],
      dtype='object')

In [19]:

columns = ['FreebaseMovieID', 'original_title', 'release_date',
       'MovieBoxOfficeRevenue', 'MovieRuntime', 'MovieLanguages',
       'MovieCountries', 'MovieGenres', 'PlotSummaries']
df_movies.columns = columns


merged_df = pd.merge(merged_df, df_movies, how= 'outer', on =['original_title', 'release_date'])
print(merged_df)
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('data/merged_file.csv', index=False)


,FreebaseMovieID,original_title,release_date,MovieBoxOfficeRevenue,MovieRuntime,MovieLanguages,MovieCountries,MovieGenres,PlotSummaries
WikipediaMovieID,,,,,,,,,
975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...","Set in the second half of the 22nd century, th..."
3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",NaN
28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",NaN
9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",A series of murders of rich young women throug...
261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}","Eva, an upper class housewife, becomes frustra..."
...,...,...,...,...,...,...,...,...,...
35228177,/m/0j7hxnt,Mermaids: The Body Found,2011,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",Two former National Oceanic Atmospheric Admini...
34980460,/m/0g4pl34,Knuckle,2011,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",{{No plot}} This film follows 12 years in the ...
9971909,/m/02pygw1,Another Nice Mess,1972,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",NaN


In [21]:
merged_df

,movieId,rating,imdb_id,tmdbId,adult,belongs_to_collection,budget,genres,homepage,id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,53519,3.559569,1028528,1991.0,False,"{'id': 339577, 'name': 'Grindhouse Collection'...",25000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",http://www.deathproof.net/,1991,...,2007-05-21,25037897.0,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A crash course in revenge,Death Proof,False,6.7,1359.0
1,54995,3.546189,1077258,1992.0,False,"{'id': 339577, 'name': 'Grindhouse Collection'...",0,"[{'id': 27, 'name': 'Horror'}, {'id': 28, 'nam...",http://www.planetterror.senator.de/,1992,...,2007-04-06,0.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Humanity's last hope... Rests on a high power ...,Planet Terror,False,6.5,1109.0
2,55063,3.663636,1093842,13241.0,False,nan,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 99, 'nam...",nan,13241,...,2007-09-07,0.0,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The truth is relative.,My Winnipeg,False,7.3,17.0
3,55069,3.834008,1032846,2009.0,False,nan,852510,"[{'id': 18, 'name': 'Drama'}]",nan,2009,...,2007-08-24,1185783.0,113.0,"[{'iso_639_1': 'ro', 'name': 'Română'}]",Released,At what moment do we begin to live?,"4 Months, 3 Weeks and 2 Days",False,7.4,158.0
4,56102,3.500000,1135489,18312.0,False,nan,0,"[{'id': 99, 'name': 'Documentary'}]",http://infowars-shop.stores.yahoo.net/endgamed...,18312,...,2007-11-01,0.0,139.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,nan,Endgame: Blueprint for Global Enslavement,False,6.7,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15404,170565,NaN,4653486,359413.0,False,"{'id': 363918, 'name': 'Aurélie Laflamme Colle...",0,"[{'id': 35, 'name': 'Comedy'}]",nan,359413,...,2015-04-24,0.0,0.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,nan,Aurélie Laflamme: Les pieds sur terre,False,7.8,5.0
15405,170797,NaN,1606207,37037.0,False,nan,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",nan,37037,...,1963-12-06,0.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"A bold man lives a double life... by day, a co...","Dr. Syn, Alias the Scarecrow",False,8.0,2.0
15406,175369,NaN,6085362,425071.0,False,nan,0,[],nan,425071,...,2016-11-11,0.0,82.0,[],Released,nan,Decanted,False,0.0,0.0
15407,175667,NaN,5990474,414453.0,False,nan,0,"[{'id': 18, 'name': 'Drama'}]",https://columbusthemovie.com,414453,...,2017-08-04,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Post Production,nan,Columbus,False,6.0,4.0


In [23]:

merged_df = pd.merge(merged_df, df_movies, how= 'outer', on =['original_title', 'release_date'])
print(merged_df)
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('data/merged_file.csv', index=False)



       movieId    rating  imdb_id   tmdbId  adult  \
0      53519.0  3.559569  1028528   1991.0  False   
1      54995.0  3.546189  1077258   1992.0  False   
2      55063.0  3.663636  1093842  13241.0  False   
3      55069.0  3.834008  1032846   2009.0  False   
4      56102.0  3.500000  1135489  18312.0  False   
...        ...       ...      ...      ...    ...   
94917      NaN       NaN      NaN      NaN    NaN   
94918      NaN       NaN      NaN      NaN    NaN   
94919      NaN       NaN      NaN      NaN    NaN   
94920      NaN       NaN      NaN      NaN    NaN   
94921      NaN       NaN      NaN      NaN    NaN   

                                   belongs_to_collection    budget  \
0      {'id': 339577, 'name': 'Grindhouse Collection'...  25000000   
1      {'id': 339577, 'name': 'Grindhouse Collection'...         0   
2                                                    nan         0   
3                                                    nan    852510   
4            